In [1]:
!python -V

Python 3.11.13


In [16]:
import pandas as pd
import pickle
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import root_mean_squared_error
from sklearn.linear_model import LinearRegression

In [17]:
import mlflow

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("nyc-taxi-experiment")

/home/wen/anaconda3/envs/mlops-hw03/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


<Experiment: artifact_location='mlflow-artifacts:/256686159311041697', creation_time=1749468683827, experiment_id='256686159311041697', last_update_time=1749468683827, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [18]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [19]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [20]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

In [21]:
target = 'duration'
y_train = df_train[target].values

In [23]:


with mlflow.start_run():
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    example_input = X_train[:1]
    mlflow.sklearn.log_model(lr, artifact_path="models_mlflow")

2025/06/09 16:29:24 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run puzzled-boar-594 at: http://localhost:5000/#/experiments/256686159311041697/runs/89ffd47edae04a1b9b64337206a04d3b
🧪 View experiment at: http://localhost:5000/#/experiments/256686159311041697


In [15]:
print(lr.intercept_)

24.776484083704368
